# Preprocess Data

In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tushare as ts
from WindPy import *
import datetime
import time

%run final_project.py

matplotlib.rcParams["figure.figsize"] = (16, 9)

In [2]:
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

### 1. Get CSI 500 constituent stocks and weights

In [3]:
# get current constituent stocks and weights of CSI 500 Index
csi_500_constituent = w.wset("indexconstituent", "date=2019-04-26;windcode=000905.SH")
csi_500_constituent = pd.DataFrame(csi_500_constituent.Data, index=csi_500_constituent.Fields)
csi_500_constituent = csi_500_constituent.T
csi_500_constituent = csi_500_constituent.drop(columns=["date"])

In [4]:
csi_500_constituent.head()

,wind_code,sec_name,i_weight
0,000006.SZ,深振业A,0.159
1,000008.SZ,神州高铁,0.26
2,000009.SZ,中国宝安,0.36
3,000012.SZ,南玻A,0.154
4,000021.SZ,深科技,0.172


In [5]:
csi_500_constituent.to_pickle("csi_500_constituent_info.gz")

### 2. Get monthly data of 500 constituent stocks from 2015-01-01 to 2019-04-26

In [ ]:
stock_data = pd.DataFrame(columns=["WINDCODE", "SEC_NAME", "INDEXCODE_SW", "INDUSTRY_SW", 
                                   "EV", "PE_TTM", "PB_MRQ", "PS_TTM", "PCF_OCF_TTM", "EV2_TO_EBITDA", 
                                   "ROE", "ROIC", "PROFITTOGR", "YOYPROFIT", "YOY_TR", "TURN", "CLOSE"])

show_time("start loop")

for code in csi_500_constituent.wind_code:
    apidata = w.wsd(code, 
              "windcode,sec_name,indexcode_sw,industry_sw,ev,pe_ttm,pb_mrq,ps_ttm,pcf_ocf_ttm,ev2_to_ebitda,roe,roic,profittogr,yoyprofit,yoy_tr,turn,close", 
              "2015-01-01", "2019-4-26", 
              "industryType=1;unit=1;Period=M;Fill=Previous,PriceAdj=F")
    df = apidata_to_df(apidata)
    stock_data = pd.concat([stock_data, df], axis=0, join="outer")
    
show_time("end loop")

In [7]:
# convert index into datetime object
stock_data["date"] = pd.to_datetime(stock_data.index, format="%Y-%m-%d")
stock_data.set_index("date", inplace=True)
# save as pickle file for convenience
stock_data.to_pickle("csi_500_data_raw.gz")

### 3. Preprocess data

In [3]:
data = pd.read_pickle("csi_500_data_raw.gz")

In [4]:
data.head()

,WINDCODE,SEC_NAME,INDEXCODE_SW,INDUSTRY_SW,EV,PE_TTM,PB_MRQ,PS_TTM,PCF_OCF_TTM,EV2_TO_EBITDA,ROE,ROIC,PROFITTOGR,YOYPROFIT,YOY_TR,TURN,CLOSE
date,,,,,,,,,,,,,,,,,
2015-01-30,000006.SZ,深振业A,801180.SI,房地产,8.77497e+09,17.4401,2.23957,3.78038,-9.34294,13.0551,None,None,None,None,None,50.8544,6.5
2015-02-27,000006.SZ,深振业A,801180.SI,房地产,8.86947e+09,17.6279,2.26369,3.82109,-9.44355,13.1681,None,None,None,None,None,16.5637,6.57
2015-03-31,000006.SZ,深振业A,801180.SI,房地产,1.188e+10,23.6112,3.03204,5.11805,-12.6489,17.7806,4.7603,3.031,15.1535,597.159,428.387,91.6881,8.8
2015-04-30,000006.SZ,深振业A,801180.SI,房地产,1.57679e+10,23.2246,3.63601,4.56375,-9.02967,22.4337,4.7603,3.031,15.1535,597.159,428.387,124.399,11.68
2015-05-29,000006.SZ,深振业A,801180.SI,房地产,2.10599e+10,31.0192,4.85632,6.09542,-12.0602,28.7671,4.7603,3.031,15.1535,597.159,428.387,102.965,15.6


In [123]:
# fill nan with data of the nearest month
for code in csi_500_constituent.wind_code:
    data[data["WINDCODE"] == code] = data[data["WINDCODE"] == code].fillna(method="ffill", axis=0)
    data[data["WINDCODE"] == code] = data[data["WINDCODE"] == code].fillna(method="bfill", axis=0)
# check if there're still nans left
pd.isnull(full_data).any()

SEC_NAME         False
INDEXCODE_SW     False
INDUSTRY_SW      False
EV               False
PE_TTM           False
PB_MRQ           False
PS_TTM           False
PCF_OCF_TTM      False
EV2_TO_EBITDA     True
ROE              False
ROIC              True
PROFITTOGR       False
YOYPROFIT        False
YOY_TR           False
TURN             False
CLOSE            False
dtype: bool

In [127]:
# locate nans
bools_1 = pd.isnull(data["EV2_TO_EBITDA"])
bools_2 = pd.isnull(data["ROIC"])
ind_1 = []
for i in range(len(bools_1)):
    if bools_1[i]:
        ind_1.append(i)
ind_2 = []
for j in range(len(bools_2)):
    if bools_2[j]:
        ind_2.append(j)
ind_1 == ind_2 # nans in EV/EBITDA and ROIC appearing from same companies

True

In [128]:
# figure out the corresponding industries
ind_code = []
for ind in ind_1:
    ind_code.append(data["INDUSTRY_SW"][ind])
print(set(ind_code)) # 10 companies in banking and non-bank financial industries have missing data

{'非银金融', '银行'}


In [143]:
# get related industries' data from Wind to calculate the industries's average EV/EBITDA and ROIC

show_time("start loading")

industry_mktcap = w.wses("1000012612000000,1000012613000000", "sec_mkt_cap_today_sum_chn", "2015-01-01", "2019-04-26", "Period=M;Fill=Previous")
industry_ebitdatosales = w.wses("1000012612000000,1000012613000000", "sec_ebitdatosales_overall_glb", "2015-01-01", "2019-04-26", "Period=M;Fill=Previous")
industry_gr = w.wses("1000012612000000,1000012613000000", "sec_gr_sum_chn", "2015-01-01", "2019-04-26", "Period=M;Fill=Previous")
industry_roic = w.wses("1000012612000000,1000012613000000", "sec_roic_avg_glb", "2015-01-01", "2019-04-26", "Period=M;Fill=Previous")

show_time("end loading")

start loading: 2019-04-27 23:16:56:863190
end loading: 2019-04-27 23:16:58:359194


In [144]:
# convert the data into dataframe
df1 = to_industry_df(industry_mktcap)
df2 = to_industry_df(industry_ebitdatosales)
df3 = to_industry_df(industry_gr)
df4 = to_industry_df(industry_roic)
# concat dataframes together
industry_data = pd.concat([df1, df2["SEC_EBITDATOSALES_OVERALL_GLB"], df3["SEC_GR_SUM_CHN"], df4["SEC_ROIC_AVG_GLB"]], 
                          axis=1, join="outer")
# convert index into datetime object
industry_data["date"] = pd.to_datetime(industry_data.index, format="%Y-%m-%d")
industry_data.set_index("date", inplace=True)
# fill nan with data of the nearest month
industry_data[industry_data["INDUSTRY_SW"] == "银行"] = industry_data[industry_data["INDUSTRY_SW"] == "银行"].fillna(method="ffill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "银行"] = industry_data[industry_data["INDUSTRY_SW"] == "银行"].fillna(method="bfill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "非银金融"] = industry_data[industry_data["INDUSTRY_SW"] == "非银金融"].fillna(method="ffill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "非银金融"] = industry_data[industry_data["INDUSTRY_SW"] == "非银金融"].fillna(method="bfill", axis=0)

In [152]:
# calculate industries' EV/EBITDA 
industry_data["EV2_TO_EBITDA"] = industry_data["SEC_MKT_CAP_TODAY_SUM_CHN"] / (industry_data["SEC_EBITDATOSALES_OVERALL_GLB"] * industry_data["SEC_GR_SUM_CHN"])
# deal with the abnormal values
for i in range(len(industry_data)):
    if np.isinf(industry_data["EV2_TO_EBITDA"][i]):
        industry_data["EV2_TO_EBITDA"][i] = np.nan
industry_data[industry_data["INDUSTRY_SW"] == "银行"] = industry_data[industry_data["INDUSTRY_SW"] == "银行"].fillna(method="ffill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "银行"] = industry_data[industry_data["INDUSTRY_SW"] == "银行"].fillna(method="bfill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "非银金融"] = industry_data[industry_data["INDUSTRY_SW"] == "非银金融"].fillna(method="ffill", axis=0)
industry_data[industry_data["INDUSTRY_SW"] == "非银金融"] = industry_data[industry_data["INDUSTRY_SW"] == "非银金融"].fillna(method="bfill", axis=0)
# drop needless columns
industry_data = industry_data.drop(columns=["SEC_MKT_CAP_TODAY_SUM_CHN", "SEC_EBITDATOSALES_OVERALL_GLB", "SEC_GR_SUM_CHN"])

In [164]:
industry_data.to_pickle("industry_data.gz")

In [168]:
# fill the nans with industry's average
for ind in ind_1: # loop through the nans
    if data["INDUSTRY_SW"][ind] == "银行": # see which industry it belongs to
        date = data.index[ind]
        for index, row in industry_data[industry_data["INDUSTRY_SW"] == "银行"].iterrows():
            if index == date:
                data["EV2_TO_EBITDA"][ind] = row["EV2_TO_EBITDA"]
                data["ROIC"][ind] = row["SEC_ROIC_AVG_GLB"]
    if data["INDUSTRY_SW"][ind] == "非银金融":
        date = data.index[ind]
        for index, row in industry_data[industry_data["INDUSTRY_SW"] == "非银金融"].iterrows():
            if index == date:
                data["EV2_TO_EBITDA"][ind] = row["EV2_TO_EBITDA"]
                data["ROIC"][ind] = row["SEC_ROIC_AVG_GLB"]

In [170]:
pd.isnull(data).any() # check if there're still nans

WINDCODE         False
SEC_NAME         False
INDEXCODE_SW     False
INDUSTRY_SW      False
EV               False
PE_TTM           False
PB_MRQ           False
PS_TTM           False
PCF_OCF_TTM      False
EV2_TO_EBITDA    False
ROE              False
ROIC             False
PROFITTOGR       False
YOYPROFIT        False
YOY_TR           False
TURN             False
CLOSE            False
dtype: bool

In [173]:
data.to_pickle("csi_500_data_preprocessed.gz")